
# Introduction to Decision Trees 
## 

## Background

Let's do a high-level recap of what we've learned in this course so far:

Say we have input data $X = (X_1, X_2, ..., X_n)$ and corresponding class labels $Y = (Y_1, Y_2, ..., Y_n)$ where $n$ represents the number of observations/instances (i.e., unique samples). Much of statistical learning concerns trying to model this relationship between our data's $X$ and $Y$. In particular, we assert that the $Y$'s were produced/generated by some underlying function $f(X)$, and that there is inevitably some noise and systematic, implicit bias and error $\epsilon$ that cannot be captured by any $f(X)$. Thus, we have:

$Y = f(X) + \epsilon$

Its important to remember that most of the time (and certainly in this class) X is often times a vector in a multi-dimensional space, because we are collecting a lot of data. This has advantages, because more features usually means better prediction.  

If you are working with labels $Y$, and you make use of them for your modelling, then you are working on a **supervised** learning task. If you do not have or make use of $Y$ values, and you are only concerned with the input data $X$, you are working on an **unsupervised** learning task.  I hope to talk about this next week, as it may be helpful in your class projects.  

Modeling data concerns either **prediction** or **inference**:

**Prediction:** concerns trying to learn a function $\hat{f}(X)$ that is as close as possible to the true function $f(X)$. This allows us to estimate $Y$ values for any new input data $X$.
*   This is what you are doing this week with Logistic Regression models to predict heart disease (the labels). 

**Inference:** concerns trying to understand/model the _relationship_ between $X$ and $Y$, effectively learning how the data was generated.  
*   This is what we did when we fit different parameters of the shifted Wald distribution to response time data that came from different experimental conditions (the labels)

### What if our data looked like this? 

![](images/complex.png)

### We could develop complicated models (e.g., deep learning neural networks to solve the problem of separating the two classes of data)
### It would be desirable to build models of data that allow for complex decision boundaries while maintaining intepretability. 

## What is a decision tree?  

### At its heart, a decision tree is a flow chart. 

### Formally, a decision tree model is one in which the final outcome of the model is based on a series of comparisons of the values of predictors against threshold values.
### In a graphical representation (flow chart),
* ### the internal nodes of the tree represent attribute testing.
* ### branching in the next level is determined by attribute value.
* ### terminal leaf nodes represent class assignments.

![](images/orangelemon.png)



### Unlike Logistic Regression, which makes lines of arbitrary orientation, in a decision tree every flow chart tree corresponds to a partition of the feature space by axis aligned lines or (hyper) planes. Conversely, every such partition can be written as a flow chart tree. 

### Given a training set, learning a decision tree model for binary classification means:
* ###  producing an optimal partition of the feature space with axis-aligned linear boundaries (maximally interpretable!),
* ### each region is predicted to have a class label based on the largest count of the training points in that region. 


### My goal is for none of the topics we learn in this class to seem like nebulus concepts or black-boxes of magic. In this course, it's important to understand the models that you can use to help you with your data, and this includes not only knowing how to invoke these as tools within Python libraries but to have an understanding of what each model is actually doing 'under the hood' -- how it actually works -- as this provides insights into how to interpret the model.  



In [ ]:
# imports
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score

In [ ]:
#Load the play_tennis data set 
tennis_df = pd.read_csv("data/play_tennis.csv")
tennis_df.info()
tennis_df.head()

In [ ]:
tennis_num = tennis_df
tennis_num = pd.get_dummies(tennis_num, columns=['outlook', 'temp', 'humidity', 'windy'])
tennis_num.info()
tennis_num.head()

class_names   = ['No','Yes']
feature_names = tennis_num.columns[1:]

In [ ]:
# separate our data into X and Y portions
x_train = tennis_num.iloc[:, tennis_num.columns != 'play'].values
y_train = tennis_num['play'].values

We can build a DecisionTree classifier as follows:

In [ ]:
dt = DecisionTreeClassifier().fit(x_train, y_train)

In [ ]:
plt.figure(figsize=(12,8))
tree_vis = plot_tree(dt, filled=True, feature_names = feature_names, class_names = class_names)

### The goal here is to make splits that will create pure as possible regions of the feature space.  The Gini coefficient is a statistical measure of how "pure" a class is.  The goal of a decision tree is to partition the data into pure classes with gini = 0 

In the above example, we did not use the tree to do any classification. Our data was too small to consider such.

### Let's revisit the PIMA Indians diabetes data.

In [ ]:
# load in the data
pima = pd.read_csv("data/diabetes.csv")
pima.info()
cols = pima.columns
diabetes = pima['Outcome']
predictors = pima[cols[1:8]]


In [ ]:
# isolate out the data we need
X             = np.array(predictors)
y             = diabetes
class_names   = class_names=['Undiagnosed','Diabetes']
feature_names = cols[1:8]

# perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# fit a decision tree classifier with max_depth=2
dt_pima = DecisionTreeClassifier(max_depth=2, random_state=42)
dt_pima.fit(X_train,y_train)

In [ ]:
plt.figure(figsize = (12,8))
plot_tree(dt_pima, filled=True, feature_names = feature_names, class_names = class_names)
plt.show()

### Precision - What proportion of postive identifications were actually correct
###  
### TP = True Positive
### FP = False Positive
### TN - True Negative 
### FN - False Negative
### Accuracy - what fraction did you get right 
### $$ Accuracy = \frac{TP+TN}{TP+FP+TN+FN}$$  
### Precision - what fraction of the positives were labeled correctly?
### $$ Precision = \frac{TP}{TP+FP}$$
### Recall - What proportion of actual positive was identified correctly? 
### $$ Recall = \frac{TP}{TP+FN}$$

In [ ]:
y_pred = dt_pima.predict(X_test)
print('accuracy score: %.2f' % accuracy_score(y_test,y_pred))
print('precision score: %.2f' % precision_score(y_test,y_pred))
print('recall score: %.2f' % recall_score(y_test,y_pred))

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
print(cnf_matrix)

In [ ]:
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="jet" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.xticks(tick_marks+0.5, class_names)
plt.yticks(tick_marks+0.5, class_names)

In [ ]:
y_pprob = dt_pima.predict_proba(X_test)
fpr, tpr, _ = roc_curve(y_test,  y_pprob[:,1])
auc = roc_auc_score(y_test, y_pprob[:,1])
plt.plot(fpr,tpr,label="DecisionTree Pima, auc="+str(auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0,1],[0,1],'k-')
plt.legend(loc=4)
plt.show()